In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, BatchNormalization
from keras.utils import to_categorical
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import pandas as pd
import os
import librosa

# Set the path to the full audio dataset 
fulldatasetpath = r'./images/'

metadata = pd.read_csv(r'./metadatas/matadata.csv')

features = []

# Iterate through each sound file and extract the features 
for index, row in metadata.iterrows():
    file_name = os.path.join(os.path.abspath(fulldatasetpath), str(row["category"]), 'Case ' + str(row["number"]) + ' ' + str(row["ID"]))
    mri = []
    for i in range(1, 29):
        im = image.load_img(file_name + r'/' + str(i) + r'.jpg', target_size = (320, 320), color_mode = 'grayscale')
        im = img_to_array(im)
        print(im.shape)
        mri.append(im)
    
    class_label = row["category"]
    features.append([mri, class_label])
# Convert into a Panda dataframe 
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])

print('Finished feature extraction from ', len(featuresdf), ' files')

(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 320, 1)
(320, 

In [4]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())
print(X[0].shape)
# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

# split the dataset 
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)



(28, 320, 320, 1)


In [9]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

a, num_channels, num_rows, num_columns, b = X.shape
print(X[0].shape)
x_train = x_train.reshape(x_train.shape[0],num_channels, num_rows, num_columns, 1)
x_test = x_test.reshape(x_test.shape[0],num_channels, num_rows, num_columns, 1)

num_labels = yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()
model.add(Conv3D(filters=8, kernel_size=3, input_shape=(num_channels, num_rows, num_columns, 1), activation='relu'))
model.add(MaxPooling3D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv3D(filters=16, kernel_size=3, activation='relu'))
model.add(MaxPooling3D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv3D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling3D(pool_size=2))
model.add(Dropout(0.2))

model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

(28, 320, 320, 1)


In [10]:

# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=1)
accuracy = 100 * score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_4 (Conv3D)            (None, 26, 318, 318, 8)   224       
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 13, 159, 159, 8)   0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 13, 159, 159, 8)   0         
_________________________________________________________________
conv3d_5 (Conv3D)            (None, 11, 157, 157, 16)  3472      
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 5, 78, 78, 16)     0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 5, 78, 78, 16)     0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 3, 76, 76, 32)    

In [13]:
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 

num_epochs = 1000
num_batch_size = 20

checkpointer = ModelCheckpoint(filepath='models/weights.best.basic_cnn1.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

333

Epoch 00901: val_loss did not improve from 0.01806
Epoch 902/1000
23/23 [==============================] - 1s 27ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 8.8117 - val_accuracy: 0.8333

Epoch 00902: val_loss did not improve from 0.01806
Epoch 903/1000
23/23 [==============================] - 1s 27ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 8.8321 - val_accuracy: 0.8333

Epoch 00903: val_loss did not improve from 0.01806
Epoch 904/1000
23/23 [==============================] - 1s 27ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 8.8303 - val_accuracy: 0.8333

Epoch 00904: val_loss did not improve from 0.01806
Epoch 905/1000
23/23 [==============================] - 1s 27ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 8.8018 - val_accuracy: 0.8333

Epoch 00905: val_loss did not improve from 0.01806
Epoch 906/1000
23/23 [==============================] - 1s 27ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 8.7895 - val_accuracy

In [14]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  1.0
Testing Accuracy:  0.8333333134651184
